In [0]:
!pip install plotly

In [0]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
idx = pd.IndexSlice

VANILLA = "#fdfaee"
GOLD = "#ffd700"
SILVER = "#c0c0c0"
BLUE = "#b4b4fa"

# Preparing data

In [0]:
# data created using these ideas: https://www.kaggle.com/camnugent/sandp500
# but for the period from Jan 1, 2019 to Dec 31, 2019, using the newer S&P 500 state

# stock prices for companies in S&P 500 
stock_prices = pd.read_csv(
    'https://git.io/JvqxQ',
    parse_dates=['date']
)

# daily prices of 
etf_prices = pd.read_csv(
    'https://git.io/Jvqx5', 
    parse_dates=['date']
)

# stock to GICS-sector mapping
stock_sectors = stock_prices[['symbol', 'sector']].drop_duplicates().set_index('symbol')['sector']

# sector to ETF mapping
sector_etfs = etf_prices[['symbol', 'sector']].drop_duplicates().dropna().set_index('sector')['symbol']

stock_prices = stock_prices.set_index(['date', 'symbol'])['close'].sort_index()
etf_prices = etf_prices.set_index(['date', 'symbol'])['close'].sort_index()

symbols = stock_sectors.index.tolist()
etfs = sector_etfs.values.tolist()
market = 'SPY'

def stock_to_etf(symbol):
    return sector_etfs[stock_sectors[symbol]]

# Figure 1: comparing the performance of stocks, sectors and the market

In [13]:
def make_trace(symbol, prices, type, visible=False):
    prices = prices.loc[idx[:, symbol]]
    returns = prices.div(prices[0]).sub(1).mul(100)

    color = GOLD if type == 'stock' else SILVER if type == 'sector' else BLUE
    x_text = 1.05 if type == 'stock' else 0.95

    return go.Scatter(
        x=returns.index,
        y=returns.values,
        visible=visible,
        opacity=0.6,
        marker=dict(color=color),
        name=symbol,
    )

# 
# add traces and annotations
stock_data = []
sector_data = []
market_data = []

# stocks
for symbol in symbols:
    visible = symbol==symbols[0]
    trace = make_trace(symbol, stock_prices, 'stock', visible)
    stock_data.append(trace)

# sector ETFs
for symbol in etfs:
    visible = symbol==stock_to_etf(symbols[0])
    trace = make_trace(symbol, etf_prices, 'sector', visible)
    sector_data.append(trace)

# S&P 500 ETF
trace = make_trace(market, etf_prices, 'market', True)
market_data.append(trace)

data = stock_data + sector_data + market_data

#
# add dropdown menu
buttons = []
for sidx, symbol in enumerate(symbols):
    sector = stock_sectors[symbol]
    etf = stock_to_etf(symbol)
    eidx = etfs.index(etf)
    v1 = [False]*len(symbols)
    v1[sidx] = True
    v2 = [False]*len(etfs)
    v2[eidx] = True
    v3 = [True]
    visible = v1 + v2 + v3

    button = dict(
        label=symbol,
        method='update',
        args=[{'visible': visible},
              {'annotations': [dict(
                     text="Symbol:",
                     xref="paper", x=-0.05, xanchor="left",
                     yref="paper", y=1.175, yanchor="middle",
                     showarrow=False,
                     font=dict(
                         family="Montserrat",
                         size=18,
                         color=VANILLA
                     )
                 ), 
                 dict(
                     text=f"Sector:      {sector}",
                     xref="paper", x=-0.05, xanchor="left",
                     yref="paper", y=1.075, yanchor="middle",
                     showarrow=False,
                     font=dict(
                         family="Montserrat",
                         size=18,
                         color=VANILLA
                     ) 
                 )],}])
    buttons.append(button)

menu = go.layout.Updatemenu(
    buttons=buttons,
    showactive=True,
    active=0,
    direction="down",
    x=0.125,
    xanchor="center",
    y=1.175,
    yanchor="middle",
    bgcolor=VANILLA,
    font=dict(
        color="darkgray",
    )
)

# 
# specify layout 
layout = dict(
    title=dict(
        text="Stock/Sector/Market performance",
        xanchor='right',
        x=0.975,
    ),
    xaxis=go.layout.XAxis(
        title='Date',
        showspikes=True,
        spikemode='across+toaxis',
        spikesnap='cursor',
        spikethickness=2,
        spikecolor=SILVER,
        spikedash='dot',
        range=["2019-01-01", "2019-12-31"],
    ),
    yaxis=go.layout.YAxis(
        title='Return, %',
        hoverformat='.1f%',
    ),
    font=dict(
        family="Montserrat",
        size=18,
        color=VANILLA
    ),
    annotations=[dict(
                     text="Symbol:",
                     xref="paper", x=-0.05, xanchor="left",
                     yref="paper", y=1.175, yanchor="middle",
                     showarrow=False,
                     font=dict(
                         family="Montserrat",
                         size=18,
                         color=VANILLA
                     )
                 ), 
                 dict(
                     text=f"Sector:      {stock_sectors[symbols[0]]}",
                     xref="paper", x=-0.05, xanchor="left",
                     yref="paper", y=1.075, yanchor="middle",
                     showarrow=False,
                     font=dict(
                         family="Montserrat",
                         size=18,
                         color=VANILLA
                     ) 
                 )],
    updatemenus=[menu],
    hovermode='x',
    spikedistance=-1,
    template='plotly_dark',
    width=1000,
    height=500,
)

fig = go.Figure(data=data, layout=layout)
fig

Output hidden; open in https://colab.research.google.com to view.

# Figure 2: Top winners and losers within a sector


In [0]:
def make_trace(sector='All'):
    if sector == 'All':
        symbols_ = symbols.copy()
    else:
        symbols_ = [s for s in symbols if stock_sectors[s] == sector]

    prices = stock_prices.loc[idx[:, symbols_]]
    returns = 100*prices.groupby('symbol').apply(lambda x: x[-1]/x[0]-1)

    winners = returns[returns > 0].nlargest(5)
    losers = returns[returns < 0].nsmallest(5)[::-1]

    trace_winners = go.Bar(
        x=winners.values,
        y=winners.index,
        orientation='h',
        opacity=0.6,
        marker=dict(color=BLUE),
        name='',
        hovertemplate="%{x:.2f}%",
        visible=(sector == 'All'),
    )

    trace_losers = go.Bar(
        x=losers.values,
        y=losers.index,
        orientation='h',
        opacity=0.6,
        marker=dict(color=GOLD),
        name='',
        hovertemplate="%{x:.2f}%",
        visible=(sector == 'All')
    )

    return [trace_winners, trace_losers]


sectors = ['All'] + sector_etfs.index.tolist()

data = []
for sector in sectors:
    data.extend(make_trace(sector))

buttons = []
for k, sector in enumerate(sectors):
    visible = [False]*2*len(sectors)
    visible[2*k] = visible[2*k+1] = True
    button = dict(
        label=sector,
        method='update',
        args=[{'visible': visible}])
    buttons.append(button)

menu = go.layout.Updatemenu(
    buttons=buttons,
    showactive=True,
    active=0,
    direction="down",
    x=0.2,
    xanchor="center",
    y=1.175,
    yanchor="middle",
    bgcolor=VANILLA,
    font=dict(
        color="darkgray",
    ),
)

layout = dict(
    title=dict(
        text="Top S&P 500 winners and losers of 2019",
        x=0.975,
        xanchor="right",
    ),
    xaxis=go.layout.XAxis(
        title="YTD price change, %",
        showgrid=True,
        gridwidth=2,
    ),
    yaxis=go.layout.YAxis(
        autorange="reversed",
    ),
    font=dict(
        family="Montserrat",
        size=18,
        color=VANILLA
    ),
    annotations=[dict(
        text="Sector:",
        xref="paper", x=-0.02,
        xanchor="center",
        yref="paper", y=1.175,
        yanchor="middle",
        showarrow=False,
        font=dict(
            family="Montserrat",
            size=18,
            color=VANILLA
        )
    )],
    template='plotly_dark',
    showlegend=False,
    barmode='overlay',
    updatemenus=[menu],
    width=1000,
    height=500,
)

fig = go.Figure(data=data, layout=layout)
fig

# Figure 3 - Sectors performance by month 

In [0]:
# daily returns
returns = etf_prices.groupby('symbol').pct_change().unstack().drop('SPY', axis=1)
order = returns.add(1).prod().sort_values().index

# monthly and quarterly returns
monthly_returns = 100*returns.add(1).groupby(pd.Grouper(freq='MS')).prod().sub(1)[order]
quarterly_returns = 100*returns.add(1).groupby(pd.Grouper(freq='QS')).prod().sub(1)[order]

months = monthly_returns.index
quarters = quarterly_returns.index
mapping = {v:k for k, v in sector_etfs.items()}

text_m = [[f"Sector: {mapping[etf]}<br />"
           f"Period: {m:%b %Y}<br />"
           f"Return: {monthly_returns.loc[m, etf]:.1f}%" 
           for m in months] for etf in order]
text_q = [[f"Sector: {mapping[etf]}<br />"
           f"Period: {q.to_period('q')},<br />"
           f"Return: {quarterly_returns.loc[q, etf]:.1f}%" 
           for q in quarters] for etf in order]

monthly = go.Heatmap(
    x=monthly_returns.index,
    y=monthly_returns.columns,
    z=monthly_returns.T,
    colorscale='thermal',
    hoverinfo='text',
    hovertext=text_m,
    opacity=0.75,
    xgap=1, ygap=1,
)

quarterly = go.Heatmap(
    x=quarterly_returns.index,
    y=quarterly_returns.columns,
    z=quarterly_returns.T,
    colorscale='thermal',
    hoverinfo='text',
    hovertext=text_q,
    visible=False,
    opacity=0.75,
    xgap=1, ygap=1,
)

data = [monthly, quarterly]

buttons = [
    dict(
        label='By month',
        method='update',
        args=[{'visible': [True, False]},
              {'xaxis': dict(
                  showgrid=False,
                  nticks=12, tickformat='%b')}]),
    dict(
        label='By quarter',
        method='update',
        args=[{'visible': [False, True]}, 
              {'xaxis': dict(
                  showgrid=False,
                  tickvals=['2019-01-01', '2019-04-01', 
                            '2019-07-01', '2019-10-01'],
                  ticktext=['Q1', 'Q2', 'Q3', 'Q4'])}])
]

menu = go.layout.Updatemenu(
    type='buttons',
    buttons=buttons,
    showactive=True,
    active=0,
    direction="right",
    x=0.925,
    xanchor="center",
    y=1.075,
    yanchor="middle",
    bgcolor=VANILLA,
    font=dict(
        color="darkgray",
    ),
)
  
layout = dict(
    title=dict(
        text='S&P 500 sectors performance in 2019',
        xanchor='right', x=0.95,
    ),
    xaxis=dict(
        nticks=12,
        tickformat='%b',
        showgrid=False,
    ),
    yaxis=dict(
        ticksuffix='  ',
        showgrid=False,
    ),
    font=dict(
        family="Montserrat",
        size=18,
        color=VANILLA
    ),
    template='plotly_dark',
    updatemenus=[menu],
    width=1000,
    height=500,
)

fig = go.Figure(data=data, layout=layout)
fig